In [3]:
import sys
!{sys.executable} -m pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=0cac82414c4e4a7c3a1b0ece85c109c3fa1d582c1ce442fde1e8a6b8d05bd8d7
  Stored in directory: /Users/aristophanesalbertusalvin/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [1]:
import numpy as np
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Loading the data and making the train-test splits.

review_files = load_files('./cleaned_op_spam', encoding='latin-1')
x_train, x_test, y_train, y_test = train_test_split(review_files.data, review_files.target, test_size=0.2)

print(review_files.target_names)
# print(review_files)

['negative deceptive', 'negative truthful', 'positive deceptive', 'positive truthful']


In [3]:
# Since we're working with text data, we need to vectorize the documents / turn each document
# into a point vector in a n-dimentions space. 
# Moreover, rather than simply using the word counts, we want to use the Term Frequency as feature value.

vectorizer = TfidfVectorizer(stop_words='english')
x_trainvec = vectorizer.fit_transform(x_train)
x_testvec = vectorizer.transform(x_test)
# print(x_trainvec.toarray())
# print(vectorizer.get_feature_names_out())

In [7]:
mnb = MultinomialNB()
y_preds1 = mnb.fit(x_trainvec, y_train).predict(x_testvec)
print("MNB correct predictions %d out of %d" % ((y_test == y_preds1).sum(), x_testvec.shape[0]))

loreg = LogisticRegression(solver='lbfgs')
y_preds2 = loreg.fit(x_trainvec, y_train).predict(x_testvec)
print("LReg correct predictions %d out of %d" % ((y_test == y_preds2).sum(), x_testvec.shape[0]))

# print('Length predictions', len(y_preds))
# print('Length y_test', len(y_test))

MNB correct predictions 250 out of 320
LReg correct predictions 248 out of 320


In [8]:
correct_pred = 0
for i in range(len(y_test)):
    if y_preds1[i] == y_test[i]:
        correct_pred += 1

print(f"Correct predictions {correct_pred} out of {len(y_test)}")

Correct predictions 250 out of 320


In [9]:
neg_dec_prob_sorted = mnb.feature_log_prob_[0, :].argsort()[::-1]
neg_tru_prob_sorted = mnb.feature_log_prob_[1, :].argsort()[::-1]
pos_dec_prob_sorted = mnb.feature_log_prob_[2, :].argsort()[::-1]
pos_tru_prob_sorted = mnb.feature_log_prob_[3, :].argsort()[::-1]

print(np.take(vectorizer.get_feature_names_out(), neg_dec_prob_sorted[:10]))
print(neg_dec_prob_sorted[:10])
print(np.take(vectorizer.get_feature_names_out(), neg_tru_prob_sorted[:10]))
print(neg_tru_prob_sorted[:10])
print(np.take(vectorizer.get_feature_names_out(), pos_dec_prob_sorted[:10]))
print(pos_dec_prob_sorted[:10])
print(np.take(vectorizer.get_feature_names_out(), pos_tru_prob_sorted[:10]))
print(pos_tru_prob_sorted[:10])

['room' 'hotel' 'chicago' 'stay' 'service' 'like' 'did' 'desk'
 'experience' 'night']
[6295 3741 1463 7086 6559 4372 2246 2209 2833 4989]
['room' 'hotel' 'night' 'stay' 'bed' 'did' 'desk' 'service' 'called'
 'told']
[6295 3741 4989 7086  913 2246 2209 6559 1267 7588]
['hotel' 'chicago' 'stay' 'great' 'room' 'staff' 'rooms' 'service'
 'definitely' 'comfortable']
[3741 1463 7086 3424 6295 7035 6298 6559 2132 1636]
['great' 'hotel' 'room' 'location' 'staff' 'stay' 'chicago' 'stayed'
 'clean' 'comfortable']
[3424 3741 6295 4437 7035 7086 1463 7087 1534 1636]


In [10]:
def run_cross_validation(nfolds, ngram=1):
    tf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(ngram, ngram))
    x_train_vec = tf_vectorizer.fit_transform(review_files.data)
    y_train_vec = review_files.target

    mnbayes = MultinomialNB()
    logreg = LogisticRegression(solver='lbfgs')
    logreg2 = LogisticRegression(solver='liblinear')
    logreg3 = LogisticRegression(solver='saga')
    dtree = DecisionTreeClassifier()
    ranForest = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
    
    # Naive Bayes
    bayes_acc = cross_val_score(mnbayes, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    bayes_prec = 0 #cross_val_score(mnbayes, x_train_vec, y_train_vec, cv=nfolds, scoring='precision').mean()
    bayes_rec = 0 #cross_val_score(mnbayes, x_train_vec, y_train_vec, cv=nfolds, scoring='recall').mean()
    
    #Logistic regression
    lg_acc = cross_val_score(logreg, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    lg_prec = 0 #cross_val_score(logreg, x_train_vec, y_train_vec, cv=nfolds, scoring='precision').mean()
    lg_rec = 0 #cross_val_score(logreg, x_train_vec, y_train_vec, cv=nfolds, scoring='recall').mean()
    
    lg2_acc = cross_val_score(logreg2, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    lg2_prec = 0 #cross_val_score(logreg2, x_train_vec, y_train_vec, cv=nfolds, scoring='precision').mean()
    lg2_rec = 0 #cross_val_score(logreg2, x_train_vec, y_train_vec, cv=nfolds, scoring='recall').mean()
    
    lg3_acc = cross_val_score(logreg3, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    lg3_prec = 0 #cross_val_score(logreg3, x_train_vec, y_train_vec, cv=nfolds, scoring='precision').mean()
    lg3_rec = 0 #cross_val_score(logreg3, x_train_vec, y_train_vec, cv=nfolds, scoring='recall').mean()
            
    dt_acc = cross_val_score(dtree, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    dt_prec = 0 #cross_val_score(dtree, x_train_vec, y_train_vec, cv=nfolds, scoring='precision').mean()
    dt_rec = 0 #cross_val_score(dtree, x_train_vec, y_train_vec, cv=nfolds, scoring='recall').mean()
    
    ranForest_acc = cross_val_score(ranForest, x_train_vec, y_train_vec, cv=nfolds, scoring='accuracy').mean()
    ranForest_prec = 0 #cross_val_score(ranForest, x_train_vec, y_train_vec, cv=nfolds, scoring='precision').mean()
    ranForest_rec = 0 #cross_val_score(ranForest, x_train_vec, y_train_vec, cv=nfolds, scoring='recall').mean()
    
    print(f'         Model                         | Accuracy | Precision | Recall')
    print(f'Naive Bayes                            | {round(bayes_acc, 3)}    | {round(bayes_prec, 3)}    | {round(bayes_rec, 3)}')
    print(f'Logistic Regression - LBFGS solver     | {round(lg_acc, 3)}    | {round(lg_prec, 3)}    | {round(lg_rec, 3)}')
    print(f'Logistic Regression - Liblinear solver | {round(lg2_acc, 3)}    | {round(lg2_prec, 3)}    | {round(lg2_rec, 3)}')
    print(f'Logistic Regression - Saga solver      | {round(lg3_acc, 3)}    | {round(lg3_prec, 3)}    | {round(lg3_rec, 3)}')
    print(f'Decision Tree                          | {round(dt_acc, 3)}    | {round(dt_prec, 3)}    | {round(dt_rec, 3)}')
    print(f'Random Forest                          | {round(ranForest_acc, 3)}    | {round(ranForest_prec, 3)}    | {round(ranForest_rec, 3)}')

In [ ]:
print('===== UNIGRAM, 5 folds cv =====')
run_cross_validation(nfolds=5, ngram=1)

print('===== UNIGRAM, 10 folds cv =====')
run_cross_validation(nfolds=10, ngram=1)

print('\n===== BIGRAM, 5 folds cv =====')
run_cross_validation(nfolds=5, ngram=2)

print('\n===== BIGRAM, 10 folds cv =====')
run_cross_validation(nfolds=10, ngram=2)

===== UNIGRAM, 5 folds cv =====
         Model                         | Accuracy | Precision | Recall
Naive Bayes                            | 0.791    | 0    | 0
Logistic Regression - LBFGS solver     | 0.817    | 0    | 0
Logistic Regression - Liblinear solver | 0.817    | 0    | 0
Logistic Regression - Saga solver      | 0.817    | 0    | 0
Decision Tree                          | 0.513    | 0    | 0
Random Forest                          | 0.593    | 0    | 0
===== UNIGRAM, 10 folds cv =====
